In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
olympic_df = pd.read_csv('Machine_Learning_datasetv2.csv')
olympic_df = olympic_df.drop(['Unnamed: 0'], axis=1)